# MongoDB Embedding Pipeline (Notebook)

This notebook demonstrates how to: 
- Install required Python packages
- Load the cleaned CSV (created by your preprocessing)
- Import the `mongo_embedding_pipeline.py` script from `python-scripts/`
- Generate embeddings for a small sample and upsert them into local MongoDB (mongodb://localhost:27017)
- Run a sample similarity query

Notes: run the cells in order. The notebook uses the pipeline implementation already added to `python-scripts/mongo_embedding_pipeline.py`.

In [13]:
%pip install --quiet sentence-transformers pymongo tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Standard imports

from sentence_transformers import SentenceTransformer
from pymongo import MongoClient, ReplaceOne
from typing import Optional
from tqdm import tqdm
import os
from pathlib import Path
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
# External libs (ensure installed via pip in previous cell)
from pymongo import MongoClient, ReplaceOne
from typing import Optional




In [15]:
# Helper: get mongo client (local by default)
def get_mongo_client(uri: Optional[str] = None) -> MongoClient:
    uri = uri or os.getenv('MONGO_URI', 'mongodb://localhost:27017')
    return MongoClient(uri)



In [16]:


class MongoNativePipeline:
    def __init__(self, mongo_uri=None, db_name='cinebot', collection_name='movies_notebook'):
        self.client = MongoClient(mongo_uri or os.getenv("MONGO_URI", "mongodb://localhost:27017"))
        self.db = self.client[db_name]
        self.coll = self.db[collection_name]
        self.model = None  # will load later

    # ------------------------
    def load_embedding_model(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """Load the embedding model (SentenceTransformers)."""
        print(f"🔄 Loading embedding model: {model_name}")
        self.model = SentenceTransformer(model_name)
        print("✅ Model loaded successfully")

    # ------------------------
    def upsert_with_embeddings(self, df: pd.DataFrame, text_col="description", id_col="id", bulk_write_size=25):
        """Upsert movie docs with embeddings generated from the description field."""
        if self.model is None:
            raise RuntimeError("❌ Model not loaded. Call load_embedding_model() first.")

        ops = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Upserting with embeddings"):
            text = str(row.get(text_col) or "")
            movie_id = str(row[id_col])
            embedding = self.model.encode(text)
            if hasattr(embedding, "tolist"):
                embedding = embedding.tolist()

            doc = {
                "_id": movie_id,
                "id": movie_id,
                "title": row.get("title"),
                "year": row.get("year"),
                "description": text,
                "embedding": embedding,  # ✅ key part
                "genres": parse_list(row.get("genres")),
                "languages": parse_list(row.get("languages")),
                "directors": parse_list(row.get("directors")),
                "stars": parse_list(row.get("stars")),
            }

            ops.append(ReplaceOne({"_id": movie_id}, doc, upsert=True))

            if len(ops) >= bulk_write_size:
                self.coll.bulk_write(ops, ordered=False)
                ops = []

        if ops:
            self.coll.bulk_write(ops, ordered=False)
        print(f"✅ Upserted {len(df)} documents with embeddings into {self.coll.full_name}")

    # ------------------------
    def create_vector_index(self, dim=384):
        """Create Atlas Vector Search index once."""
        from pymongo.errors import OperationFailure

        index_name = "movie_vector_index"
        index_definition = {
            "name": index_name,
            "type": "vectorSearch",
            "definition": {
                "fields": [
                    {"type": "vector", "path": "embedding", "numDimensions": dim, "similarity": "cosine"}
                ]
            },
        }

        try:
            result = self.db.command("createSearchIndexes", self.coll.name, indexes=[index_definition])
            print("✅ Vector search index created successfully:", result)
        except OperationFailure as e:
            if "already exists" in str(e):
                print("ℹ️ Vector index already exists — skipping.")
            else:
                raise

    # ------------------------
    def search_similar(self, query, top_k=5):
        """Run MongoDB-native $vectorSearch query."""
        if not self.model:
            raise RuntimeError("❌ Model not loaded. Run load_embedding_model() first.")

        query_vec = self.model.encode(query)
        if hasattr(query_vec, "tolist"):
            query_vec = query_vec.tolist()

        pipeline = [
            {
                "$vectorSearch": {
                    "index": "movie_vector_index",  # ✅ match create_vector_index name
                    "path": "embedding",
                    "queryVector": query_vec,
                    "numCandidates": 200,
                    "limit": top_k,
                }
            },
            {
                "$project": {
                    "title": 1,
                    "year": 1,
                    "description": 1,
                    "_id": 0,
                    "score": {"$meta": "vectorSearchScore"},
                }
            },
        ]

        results = list(self.coll.aggregate(pipeline))
        return results


In [17]:
# --- Full demo: load CSV, upsert a small sample, run search, and optionally run full upload ---


In [18]:
csv_path = Path('../cleaned_database/cleaned_final_dataset3.csv')
if not csv_path.exists():
    raise FileNotFoundError(f'Cleaned CSV not found at: {csv_path.resolve()}')
df = pd.read_csv(csv_path)
print('Loaded dataframe shape:', df.shape)

Loaded dataframe shape: (63249, 25)


In [ ]:
load_dotenv()

In [ ]:

MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB_NAME = os.getenv("MONGO_DB_NAME", "cinebot")
MONGO_COLLECTION = os.getenv("MONGO_COLLECTION", "movies_notebook")

print("✅ Connected to:", MONGO_URI.split('@')[-1])  # Just prints domain, not credentials


In [19]:
# Upload first 5 rows (small sanity check)
sample_df = df.head(5).copy()

p.upsert_from_df(sample_df)



KeyboardInterrupt



In [ ]:
# Upsert your cleaned movie DataFrame
upsert_with_embeddings_in_batches(p, df, batch_size=300)

OperationFailure: Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs., full error: {'ok': 0.0, 'errmsg': 'Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs.', 'code': 31082, 'codeName': 'SearchNotEnabled'}

True

✅ Connected to: cinebot.cqwj7t2.mongodb.net/


In [ ]:
# Create vector index once (only needed the first time)
p.create_vector_index(dim=1536)  # use your provider’s embedding dimension


OperationFailure: Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs., full error: {'ok': 0.0, 'errmsg': 'Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs.', 'code': 31082, 'codeName': 'SearchNotEnabled'}

AttributeError: 'MongoNativePipeline' object has no attribute 'load_model'

In [7]:
# Generate embeddings for a small sample (first 5 rows) to validate everything works
sample = df.head(5).copy()
docs = sample['docs'].fillna('').astype(str).tolist()
embs = p.generate_embeddings(docs, batch_size=8)
print('Generated embeddings shape:', embs.shape)
print('First embedding length:', len(embs[0]))

Generated embeddings shape: (5, 384)
First embedding length: 384


In [8]:
# Upsert the small sample into MongoDB (safe to re-run)
p.upsert_from_df(sample, csv_id_col='id', batch_size=5)
print('Upsert completed for sample collection:', p.coll.full_name)


Detected embedding dimension: 384
Upserted rows 1-5: upserted=0, modified=0
Could not create vector search index (server may not support it): Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs., full error: {'ok': 0.0, 'errmsg': 'Using Atlas Search Database Commands and the $listSearchIndexes aggregation stage requires additional configuration. Please connect to Atlas or an AtlasCLI local deployment to enable. For more information on how to connect, see https://dochub.mongodb.org/core/atlas-cli-deploy-local-reqs.', 'code': 31082, 'codeName': 'SearchNotEnabled'}
Processed 5/5 rows
All batches processed.
Upsert completed for sample collection: cinebot.movies_notebook
Upserted rows 1-5: upserted=0, modified=0
Could not create vector search index (server ma

In [9]:
# Run a sample similarity search
results = p.search_similar('action movie with superheroes', top_k=5)
print('Top results:')
if not results:
    print('No results returned. Possible reasons: no documents in collection, embeddings missing, or text filter matched nothing.')
else:
    import pandas as _pd
    rows = []
    for rank, r in enumerate(results, start=1):
        rows.append({
            'rank': rank,
            'id': r.get('_id'),
            'title': r.get('title'),
            'year': r.get('year'),
            'rating': r.get('rating'),
            'similarity': r.get('similarity'),
            'description_snippet': (r.get('description') or '')[:200]
        })
    df_results = _pd.DataFrame(rows)
    display(df_results)
    print('\nReadable list:')
    for _, row in df_results.iterrows():
        print(f"{int(row['rank'])}. {row['title']} (id={row['id']}) - similarity={row['similarity']:.4f}")
        print('   ', row['description_snippet'])


Top results:


,rank,id,title,year,rating,similarity,description_snippet
0,1,tt29195117,Justice League: Crisis on Infinite Earths - Pa...,2024,6.2,0.458105,"The Anti-Monitor, the Monitor's evil counterpa..."
1,2,tt6263850,Deadpool & Wolverine,2024,7.6,0.455439,Deadpool is offered a place in the Marvel Cine...
2,3,tt0347167,The Hero: Love Story of a Spy,2003,5.5,0.428149,Arun is an Indian undercover agent enlisted to...
3,4,tt19500164,Nimona,2023,7.5,0.411302,When a knight in a futuristic medieval world i...
4,5,tt0123248,Run Chrissie Run!,1984,5.5,0.409940,The action moves along quickly in this made-fo...


Top results:


,rank,id,title,year,rating,similarity,description_snippet
0,1,tt29195117,Justice League: Crisis on Infinite Earths - Pa...,2024,6.2,0.458105,"The Anti-Monitor, the Monitor's evil counterpa..."
1,2,tt6263850,Deadpool & Wolverine,2024,7.6,0.455439,Deadpool is offered a place in the Marvel Cine...
2,3,tt0347167,The Hero: Love Story of a Spy,2003,5.5,0.428149,Arun is an Indian undercover agent enlisted to...
3,4,tt19500164,Nimona,2023,7.5,0.411302,When a knight in a futuristic medieval world i...
4,5,tt0123248,Run Chrissie Run!,1984,5.5,0.409940,The action moves along quickly in this made-fo...



Readable list:
1. Justice League: Crisis on Infinite Earths - Part One (id=tt29195117) - similarity=0.4581
    The Anti-Monitor, the Monitor's evil counterpart, is released into the DC Multiverse and begins to destroy the different Earths that compose it. The Monitor attempts to recruit heroes from across the 
2. Deadpool & Wolverine (id=tt6263850) - similarity=0.4554
    Deadpool is offered a place in the Marvel Cinematic Universe by the Time Variance Authority, but instead recruits a variant of Wolverine to save his universe from extinction.
3. The Hero: Love Story of a Spy (id=tt0347167) - similarity=0.4281
    Arun is an Indian undercover agent enlisted to stop ISI chief Ishaq Khan. The village girl Reshma helps him but endangers her own life.
4. Nimona (id=tt19500164) - similarity=0.4113
    When a knight in a futuristic medieval world is framed for a crime he didn't commit, the only one who can help him prove his innocence is Nimona -- a mischievous teen who happens to be a shap

In [10]:
# Truncate & run full pipeline (CAUTION)
# Set confirm_run_full = True to actually delete and upload entire dataset
confirm_run_full = True  # change to True when you are ready
if confirm_run_full:
    print('Truncating collection:', p.coll.full_name)
    p.coll.delete_many({})
    print('Collection cleared. Starting full upload...')
    # Choose a batch size appropriate for your machine
    p.upsert_from_df(df, csv_id_col='id', batch_size=128)
    print('Full upload complete.')
else:
    print('confirm_run_full is False; full upload skipped. Set confirm_run_full = True to run it.')


Truncating collection: cinebot.movies_notebook
Collection cleared. Starting full upload...
Detected embedding dimension: 384
Collection cleared. Starting full upload...
Detected embedding dimension: 384


Truncating collection: cinebot.movies_notebook
Collection cleared. Starting full upload...
Detected embedding dimension: 384
Collection cleared. Starting full upload...
Detected embedding dimension: 384


KeyboardInterrupt: 

In [9]:
# Cleanup: close the MongoDB client when done
p.client.close()
print('Closed MongoDB client')

Closed MongoDB client
